# Exp4: 基于K-近邻的车牌号识别

## 一、案例简介

图像的智能处理一直是人工智能领域广受关注的一类技术，代表性的如人脸识别与 CT 肿瘤识别，在人工智能落地的进程中发挥着重要作用。其中车牌号识别作为一个早期应用场景，已经融入日常生活中，为我们提供了诸多便利，在各地的停车场和出入口都能看到它的身影。车牌号识别往往分为字符划分和字符识别两个子任务，本案例我们将关注字符识别的任务，尝试用 K-NN 的方法对分割好的字符图像进行自动识别和转化。

## 二、作业说明

### 基本要求
* 完成数据的读入和表示，将图片表示成向量并和 label 对应上；
* 构建 K-NN 模型（可调库）对测试集中的图片进行预测并计算准确率；
* 分析当 K 取不同值时测试准确率的变化。

### 扩展要求
* 分析不同距离度量方式对模型效果的影响；
* 对比平权和加权 K-NN 的效果；
* 分析训练集大小对测试结果的影响。

## 三、数据概览
本次我们使用已经分割好的车牌图片作为数据集，包括数字 0-9、字母 A-Z（不包含 O 和 I）以及省份简称共 65 个类，编号从 0 到 64。数据已经分成了训练集和测试集，里面的文件夹用 label 编号命名，一个文件夹下的所有图片都属于该文件夹对应的类，每个图片都是 20 * 20 的二值化灰度图。

下面演示一下如何借助 PIL 库将图片转化为向量：

In [1]:
from PIL import Image
img = Image.open('data/train/0/4-3.jpg')  # 打开图片
img  # 显示图片from PIL import Image
img = Image.open('data/train/0/4-3.jpg')  # 打开图片
img  # 显示图片

In [2]:
import numpy as np
pixels = np.array(img)  # 转化为 numpy 矩阵
pixels.shape

(20, 20)

## 四、模型构建

### 1. 读入数据、向量表示、对应label

In [3]:
import pandas as pd
import os

def load_data(path):
    result = []
    for label in os.listdir(path):
        if not os.path.isdir(os.path.join(path, label)):
            continue
        for file in os.listdir(os.path.join(path, label)):
            if file.endswith('.jpg'):
                result.append([label, np.array(Image.open(os.path.join(path, label, file)))])
    return pd.DataFrame(result, columns=['label', 'file'])

train_data = load_data('data/train')
test_data = load_data('data/test')

### 2. 构建KNN模型

In [7]:
print(train_data.head())
print(test_data.head())

  label                                               file
0    61  [[2, 1, 0, 0, 1, 0, 1, 1, 0, 5, 1, 0, 4, 0, 5,...
1    61  [[1, 0, 1, 0, 2, 254, 255, 128, 0, 0, 3, 0, 12...
2    61  [[0, 0, 0, 3, 0, 1, 2, 0, 0, 1, 0, 1, 0, 20, 0...
3    61  [[2, 2, 2, 0, 4, 0, 0, 4, 1, 0, 2, 0, 0, 0, 2,...
4    61  [[1, 0, 1, 0, 0, 26, 254, 126, 0, 2, 0, 77, 25...
  label                                               file
0    61  [[3, 0, 1, 5, 0, 0, 240, 254, 253, 2, 4, 28, 2...
1    61  [[1, 1, 0, 57, 254, 255, 0, 1, 0, 250, 255, 0,...
2    61  [[1, 0, 0, 4, 2, 3, 0, 2, 0, 2, 1, 0, 6, 0, 1,...
3    61  [[0, 3, 0, 1, 0, 2, 0, 0, 0, 0, 4, 0, 3, 6, 0,...
4    50  [[0, 2, 1, 6, 1, 0, 38, 0, 1, 0, 73, 158, 36, ...


In [11]:
train_data['flatten_file'] = train_data.file.map(lambda file: file.flatten())
test_data['flatten_file'] = test_data.file.map(lambda file: file.flatten())
print(train_data.head())
print(test_data.head())

  label                                               file  \
0    61  [[2, 1, 0, 0, 1, 0, 1, 1, 0, 5, 1, 0, 4, 0, 5,...   
1    61  [[1, 0, 1, 0, 2, 254, 255, 128, 0, 0, 3, 0, 12...   
2    61  [[0, 0, 0, 3, 0, 1, 2, 0, 0, 1, 0, 1, 0, 20, 0...   
3    61  [[2, 2, 2, 0, 4, 0, 0, 4, 1, 0, 2, 0, 0, 0, 2,...   
4    61  [[1, 0, 1, 0, 0, 26, 254, 126, 0, 2, 0, 77, 25...   

                                        flatten_file  
0  [2, 1, 0, 0, 1, 0, 1, 1, 0, 5, 1, 0, 4, 0, 5, ...  
1  [1, 0, 1, 0, 2, 254, 255, 128, 0, 0, 3, 0, 125...  
2  [0, 0, 0, 3, 0, 1, 2, 0, 0, 1, 0, 1, 0, 20, 0,...  
3  [2, 2, 2, 0, 4, 0, 0, 4, 1, 0, 2, 0, 0, 0, 2, ...  
4  [1, 0, 1, 0, 0, 26, 254, 126, 0, 2, 0, 77, 253...  
  label                                               file  \
0    61  [[3, 0, 1, 5, 0, 0, 240, 254, 253, 2, 4, 28, 2...   
1    61  [[1, 1, 0, 57, 254, 255, 0, 1, 0, 250, 255, 0,...   
2    61  [[1, 0, 0, 4, 2, 3, 0, 2, 0, 2, 1, 0, 6, 0, 1,...   
3    61  [[0, 3, 0, 1, 0, 2, 0, 0, 0, 0, 4, 0, 3,

In [38]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# 准备数据
X_train = np.stack(train_data.flatten_file.values)
y_train = train_data.label.values
X_test = np.stack(test_data.flatten_file.values) 
y_test = test_data.label.values

# 训练模型
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

# 预测
y_pred = knn.predict(X_test)  # 现在使用正确的X_test

# 评估
accuracy = accuracy_score(y_test, y_pred)
print(f'准确率: {accuracy:.4f}')

准确率: 0.7031
